# Fine-tuned model

This notebook:
1. Takes .jsonl fine-tuning script.
1. Provides command to run .jsonl fine-tuning.
1. Saves name of the fine-tuned model.

Then, it runs a similar procedure to other notebooks, where it:
1. Takes testing problems from /test/MATH.
1. Runs function "answer" on them, which **calls the fine-tuned DaVinci GPT-3**.
1. Saves the result in **results-finetuned-davinci** under "gpt-answer".

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\n###\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=[" END"]` so that the generated texts ends at the expected place.

# Using Fine-Tuned Model Results

In [1]:
import os
import json
import openai

with open('keys.json') as f:
    data = json.load(f)
openai.api_key = data['openai-key']

In [4]:
with open('prompts.json') as f:
    data = json.load(f)
prompt_json = data['no_wolfram']

# Calls DaVinci GPT-3 API on input text & returns output.
def answer(problem):

    full_message = "[Problem] " + problem + "\n\n" + prompt_json + "\n\n###\n\n"
    
    # Use the OpenAI API to generate a response
    response = openai.Completion.create(
        engine="davinci:ft-personal-2023-04-25-04-13-46",
        prompt=full_message,
        temperature=0, # temperature 0 keeps it deterministic
        max_tokens = 600,
        stop=[" END"]
    )

    text_output = response["choices"][0]["text"]
    print(text_output)
    return text_output

In [7]:
# Test code block: run this to test that davinci works, before you call GPT-3 API and potentially waste credits. Will print twice.
answer("What is 4+5?")

 [Problem] What is 4+5?

\begin{align*}
\ 4+5 &= \boxed{9} \\
\end{align*}


' [Problem] What is 4+5?\n\n\\begin{align*}\n\\ 4+5 &= \\boxed{9} \\\\\n\\end{align*}'

In [10]:
# Define the paths to the data directories
data_dir = "test/MATH"
modified_data_dir = "results-finetuned-davinci/MATH"

# Loop over the directories | possible: ["train", "test"]:
for data_type in ["test"]:
    # Loop over the subdirectories in the data directory
    for sub_dir in os.listdir(os.path.join(data_dir, data_type)):
        
        # Skip any files that are not directories
        if not os.path.isdir(os.path.join(data_dir, data_type, sub_dir)):
            continue
        
        # Create the corresponding subdirectory in the modified data directory
        os.makedirs(os.path.join(modified_data_dir, data_type, sub_dir), exist_ok=True)

        # Get a list of all the JSON files in the original subdirectory
        json_files = [f for f in os.listdir(os.path.join(data_dir, "test", sub_dir)) if f.endswith(".json")]
        
        # Loop over the selected JSON files in the original subdirectory
        for file_name in json_files:
            
            # Load the JSON file
            with open(os.path.join(data_dir, data_type, sub_dir, file_name), "r") as file:
                data = json.load(file)
            
            # Extract the problem field
            problem = data["problem"]
            print("Problem:", problem)

            # Call the answer function
            answer_text = answer(problem)
            
            # Add the answer field to the JSON data
            data["gptanswer"] = answer_text
            
            # Save the modified JSON data to a new file
            with open(os.path.join(modified_data_dir, data_type, sub_dir, file_name), "w") as file:
                json.dump(data, file)

Problem: What is $\dbinom{n}{n}$ for any positive integer $n$?
 [Problem] What is $\dbinom{n}{n}$ for any positive integer $n$?

[Answer] $\dbinom{n}{n} = \boxed{n^n}$.
Problem: How many paths are there from $A$ to $B$ on the lines of the grid shown, if every step must be up or to the right?[asy]size(3cm,3cm);int w=6;int h=3;int i;for (i=0; i<h; ++i){ draw((0,i) -- (w-1,i));}for (i=0; i<w; ++i){ draw((i, 0)--(i,h-1));}label("$B$", (w-1,h-1), NE);label("$A$", (0,0), SW);[/asy]
For example, here is one such valid path highlighted in blue:
[asy]
size(3cm,3cm);
int w=6;
int h=3;
int i;
for (i=0; i<h; ++i){ 
draw((0,i) -- (w-1,i));
}for (i=0; i<w; ++i){ 
draw((i, 0)--(i,h-1));
}
label("$B$", (w-1,h-1), NE);
label("$A$", (0,0), SW);

draw((0,0)--(1,0), blue+1.5);
draw((1,0)--(1,1), blue+1.5);
draw((1,1)--(2,1), blue+1.5);
draw((2,1)--(3,1), blue+1.5);
draw((3,1)--(4,1), blue+1.5);
draw((4,1)--(4,2), blue+1.5);
draw((4,2)--(5,2), blue+1.5);
[/asy]
 There are $6 \times 6 = 36$ squares on the g